<a href="https://colab.research.google.com/github/nsmancini/odsc_stuff/blob/main/network_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Packages
Note that `plotly` is only used for plotting the map. If you don't want to install it, simply comment `pip install plotly` and make sure not to run the sections that plot the input or output maps.

In [1]:
%pip install gurobipy
%pip install pandas
%pip install openpyxl
%pip install plotly

     |████████████████████████████████| 11.5 MB 6.7 MB/s 


# For Google Colab Only
If you like to run the notebook in Google Colab, follow these steps:
- Go to https://colab.research.google.com/github/decision-spot/net_opt/blob/main/network_optimization.ipynb
This should open up the notebook in Google Colab. Since there are helper modules in this repo and they are used in the main file, you should have access to all of them or you'll get `ImportError` when trying to import them. 
- To get all the files, run the following cells to clone to repo and change the current working directory path.

In [2]:
!git clone https://github.com/decision-spot/net_opt.git

Cloning into 'net_opt'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 46 (delta 22), reused 37 (delta 15), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [3]:
import os
from google.colab import files
os.chdir('net_opt')

# Load Packages

In [4]:
"""
Author = Ehsan Khodabandeh
Copyright 2022, Decision Spot, LLC

Having a set of plants and customers, decide where to open some warehouses and 
assign customers to warehouses considering a limit on the number of warehouses to open.
Plant locations are used as warehouse locations candidates.
Some plant locations must be used and some cannot be used.
The solution is driven by weighted distance or transportation cost.
"""
import sys

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from plots import plot_network
from utils import calculate_distance_haversine, prepare_location_dataframe

# Problem Definition

- We have a set of customers and a set of candidate locations for warehouses
- We need to satisfy all customers' demands
- We can assign each customer to only one warehouse
- We can only use certain number of warehouses
- It's best to assign customers to warehouses that are closer to them. That way we minimize our transportation time and cost.

## Problem Formulation

**Sets**
- $I\quad$: Set of warehouses
- $J\quad$: Set of customers

**Parameters**
- $q_j\quad$: demand of customer $j$
- $d_{ij}\quad$: distance between warehouse $i$ and customer $j$
- $c_{ij}\quad$: transportation cost between warehouse $i$ and customer $j$
- $P\quad$: maximum number of warehouses that can be used

**Variables**
- $x_i\quad$: 1 if a warehouse is opened at location $i$, 0 otherwise
- $y_{ij}\quad$: 1 if warehouse $i$ is assigned to customer $j$, 0 otherwise

\begin{align}
&\min &\sum_{ij} q_{j}d_{ij}y_{ij}&\\
&\mbox{s.t: }
&\sum_{i} y_{ij} = 1 &\quad \forall j \in J\\
&&\sum_{i} x_{i} \le P&\\
&&y_{ij} \le x_{i} &\quad \forall i \in I, j \in J\\
&&x_{i}, y_{ij} \in \{0,1\}&\quad \forall i \in I, j \in J\\
\end{align}

# Load Input Data

In [5]:
def prep_data(file_name):
    input_df_dict = pd.read_excel(file_name, sheet_name=None)
    cust_df = input_df_dict['Customers']
    plant_df = input_df_dict['Plants']
    cust_df.rename(columns={'ID': 'Customer ID', 'Name': 'Customer Name'}, inplace=True)
    plant_df.rename(columns={'ID': 'Plant ID', 'Name': 'Plant Name'}, inplace=True)
    return plant_df, cust_df

### Note:
If you don't have a gurobi license, change the `file_name` parameter to `'Small Sample Data.xlsx'` to ensure that you can run the notebook with the gurobi restricted license. Note that you still need to install the required packages.

In [6]:
# Params
max_plants = 3  # P or max number of warehouses to open
cost_per_mile = 2  # It is used when transportation cost needs to be calculated
min_cost = 450  # It is used when transportation cost needs to be calculated
open_map_in_cell = True  # This is for Jupyter Notebook
file_name = 'Small Sample Data.xlsx'  # Two choices: 'Sample Data.xlsx' and 'Small Sample Data.xlsx'
plant_df, cust_df = prep_data(file_name)

In [7]:
plant_df.iloc[:,:8].head()

,Plant ID,Plant Name,City,State,Zip Code,Country,Latitude,Longitude
0,1,Salt Lake City Plant,Salt Lake City,UT,84101,US,40.755074,-111.898490
1,2,Oklahoma City Plant,Oklahoma City,OK,73101,US,35.467763,-97.520977
2,3,Jackson Plant,Jackson,MS,39201,US,32.289153,-90.184021
3,4,Denver Plant,Denver,CO,80201,US,39.750710,-104.996225
4,5,St. Louis Plant,St. Louis,MO,63101,US,38.631358,-90.192246


In [8]:
cust_df.head()

,Customer ID,Customer Name,City,State,Zip Code,Country,Latitude,Longitude,Demand
0,1,Buffalo Customer,Buffalo,NY,14201,US,42.896219,-78.884649,1184
1,2,Fresno Customer,Fresno,CA,93701,US,36.749611,-119.786244,888
2,3,Norfolk Customer,Norfolk,VA,23501,US,36.852105,-76.292507,1564
3,4,Springfield Customer,Springfield,IL,62701,US,39.800060,-89.647829,611
4,5,Minneapolis Customer,Minneapolis,MN,55401,US,44.985775,-93.270165,2812


In [9]:
# Prepare data to plot the input network
locations = prepare_location_dataframe(plant_df, cust_df)

# Plot Input Map

In [10]:
plot_network(locations, add_path=False, auto_open_map=False, title='Input Map',
             open_map_in_cell=open_map_in_cell)

# Network Optimization

## Set up Data

In [11]:
def get_distance(orig_df, dest_df):
    df = orig_df.merge(dest_df, how='cross').set_index(['Plant ID', 'Customer ID'])
    df['Distance'] = df.apply(lambda r: calculate_distance_haversine(
        r['Latitude_x'], r['Longitude_x'], r['Latitude_y'], r['Longitude_y']), axis=1)
    return df

In [12]:
dist = get_distance(plant_df, cust_df)
dmd = cust_df.set_index(['Customer ID'])

# Sets
plnt = plant_df['Plant ID'].unique()  # I
cust = cust_df['Customer ID'].unique() # J
ij_set = set((i, j) for i in plnt for j in cust)  # Pair of I,J. Created to avoid repetition in the code

## Define Optimization Model

In [13]:
# Model
mdl = gp.Model('net_optimization')

# Variables
x = mdl.addVars(plnt, vtype=GRB.BINARY, name='x')
y = mdl.addVars(plnt, cust, vtype=GRB.BINARY, name='y')

# Constraints
# 1. Every customer must be assigned to one plant
# sum_{i} y_{ij} = 1    for all j
mdl.addConstrs((y.sum('*', j) == 1 for j in cust), 'cust_coverage')

# 2. We can use maximum P plants
# sum_{i} x_{i} <= P
mdl.addConstr(x.sum() <= max_plants, 'max_num_plants')

# 3. Cant serve a customer from a plant if it's not used
# y_{ij} <= x_{i}    for all i, j
mdl.addConstrs((y[i, j] <= x[i] for (i, j) in ij_set), 'rel_x_y')

# Objective function: minimize total weighted distance
# sum_{ij} q_{j}*d_{ij}*y_{ij}
total_weighted_dist = gp.quicksum(dist.loc[i, j]['Distance'] * dmd.loc[j]['Demand'] * y[i, j]
                                  for (i, j) in ij_set)
mdl.setObjective(total_weighted_dist, GRB.MINIMIZE)
mdl.setParam(GRB.Param.OutputFlag, 1)  # enables or disables solver output
mdl.optimize()
status = mdl.status
if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('The model is either infeasible or unbounded!')
    sys.exit(1)
elif status == GRB.OPTIMAL:
    print(f'The solution is optimal and the objective value is: {mdl.objVal:,.0f}')
else:
    print(f'Status is {status}. Investigate!')
    sys.exit(1)

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 1601 rows, 1515 columns and 4515 nonzeros
Model fingerprint: 0x5be13ee6
Variable types: 0 continuous, 1515 integer (1515 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve time: 0.02s
Presolved: 1601 rows, 1515 columns, 4515 nonzeros
Variable types: 0 continuous, 1515 integer (1515 binary)
Found heuristic solution: objective 1.149150e+08

Root relaxation: objective 6.223127e+07, 577 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6.223127e+07 6.2231e+07  0.00%     -

# Generate Outputs

In [14]:
def generate_outputs(plant_df, cust_df, dist, dmd, total_weighted_dist, x, y):
    print('=' * 40)
    assigned_list = [k for k, v in y.items() if v.x > 0.5]
    assigned_df = pd.DataFrame(assigned_list, columns=['Plant ID', 'Customer ID'])
    _opt_plants = [plant_df.loc[plant_df['Plant ID'] == k, 'City'].iloc[0]
                   for k, v in x.items() if v.x > 0.5]
    print(f'Selected plants are in: {", ".join(str(_) for _ in _opt_plants)}')
    adf = pd.merge(assigned_df, dmd[['Demand']], how='inner',
                   left_on='Customer ID', right_index=True)
    adf = pd.merge(adf.set_index(['Plant ID', 'Customer ID']), dist[['Distance']],
                   how='inner', left_index=True, right_index=True).reset_index()
    adf['Within400'] = [True if x <= 400 else False for x in adf['Distance']]
    adf['Within800'] = [True if x <= 800 else False for x in adf['Distance']]
    adf['Within1200'] = [True if x <= 1200 else False for x in adf['Distance']]
    total_dmd = adf['Demand'].sum()
    weighted_avg_dist = total_weighted_dist.getValue() / total_dmd
    print(f'Weighted Average Distance: {weighted_avg_dist:,.1f}')
    pct_dmd_within400 = adf.loc[adf['Within400'], 'Demand'].sum() / total_dmd
    pct_dmd_within800 = adf.loc[adf['Within800'], 'Demand'].sum() / total_dmd
    pct_dmd_within1200 = adf.loc[adf['Within1200'], 'Demand'].sum() / total_dmd
    print(f'Total demand: {total_dmd:,.0f}')
    print(f'Percentages of demand within 400 miles of a plant: {pct_dmd_within400 * 100:,.2f}%')
    print(f'Percentages of demand within 800 miles of a plant: {pct_dmd_within800 * 100:,.2f}%')
    print(f'Percentages of demand within 1200 miles of a plant: {pct_dmd_within1200 * 100:,.2f}%')
    print('=' * 40)
    # Create the set of lanes
    lanes = pd.merge(assigned_df, dmd[['Demand']], how='inner',
                     left_on='Customer ID', right_index=True)
    lanes['Lane'] = lanes['Plant ID'].map(str) + '-' + lanes['Customer ID'].map(str)
    lanes = lanes[['Plant ID', 'Customer ID', 'Demand', 'Lane']].merge(
        cust_df[['Customer ID', 'Customer Name', 'Latitude', 'Longitude']], on='Customer ID')
    lanes.rename(columns={'Latitude': 'Dest Lat', 'Longitude': 'Dest Lon'}, inplace=True)
    lanes = lanes.merge(plant_df[['Plant ID', 'Plant Name', 'Latitude', 'Longitude']], on='Plant ID')
    lanes.rename(columns={'Latitude': 'Origin Lat', 'Longitude': 'Origin Lon',
                          'Plant Name': 'Origin', 'Customer Name': 'Destination'}, inplace=True)
    lanes = lanes[['Lane', 'Origin', 'Destination', 'Demand', 'Plant ID',
                   'Origin Lat', 'Origin Lon', 'Customer ID', 'Dest Lat', 'Dest Lon']].drop_duplicates()
    return lanes

In [15]:
lanes = generate_outputs(plant_df, cust_df, dist, dmd, total_weighted_dist, x, y)

Selected plants are in: Phoenix, Louisville, New York City
Weighted Average Distance: 367.9
Total demand: 169,160
Percentages of demand within 400 miles of a plant: 67.39%
Percentages of demand within 800 miles of a plant: 90.62%
Percentages of demand within 1200 miles of a plant: 100.00%


## Plot Output Map

In [16]:
plot_network(lanes, auto_open_map=False, title='Solution Map', open_map_in_cell=open_map_in_cell)

# Additional Constraints

- Users ask for flexibility of inclusion and exclusion of plants from the model
- To handle this scenario, we add two more columns to "Plants" table; "Must Use" and "Can Use". 
- "Must Use" is to indicate whether a plant location must be used as a candidate. By default, there is no hard restriction for any plant.
- "Can Use" is to indicate whether a plant location can or cannot be used as a candidate. By default, all plant locations can be used as a candidate location.

We want to exclude one of the optimal locations from the solution. So, we need to set the "Can Use" value of that location to False.

Let's change that for "New York City" in the input data.

In [17]:
plant_df.loc[plant_df['City'] == 'New York City', 'Can Use'] = False
plant_df.iloc[10:25,:]  # investigate to make sure the change has happened

,Plant ID,Plant Name,City,State,Zip Code,Country,Latitude,Longitude,Must Use,Can Use
10,11,Memphis Plant,Memphis,TN,37501,US,35.033731,-89.934319,False,True
11,12,Detroit Plant,Detroit,MI,48201,US,42.346577,-83.059731,False,True
12,13,Boston Plant,Boston,MA,2101,US,42.360970,-71.053440,False,True
13,14,Atlanta Plant,Atlanta,GA,30301,US,33.753693,-84.389544,False,True
14,15,New York City Plant,New York City,NY,10001,US,40.750120,-73.997846,False,False


We need two additional sets of constraints:
- If a plant _must be used_, then variable $x_i$ for that plant must take a value of 1.
- If a plant _cannot be used_, then variable $x_i$ for that plant must take a value of 0.

Let's show _Must Use_ (or forced) plant locations with $FI$ set and _Cannot Use_ (or unavailable) plant locations with $UI$ set. The two new sets of constraints are:

\begin{align}
&&x_{i} = 1 &\quad \forall i \in FI\\
&&x_{i} = 0 &\quad \forall i \in UI\\
\end{align}

In [18]:
must_use_plants = plant_df.loc[plant_df['Must Use'], 'Plant ID'].to_list()  # Forced locations or FI set
unavailable_plants = plant_df.loc[~plant_df['Can Use'], 'Plant ID'].to_list()  # Unavailable locations or UI set

# 4. Plants that we must use
# x_{i} = 1    for all i in FI
mdl.addConstrs((x[i] == 1 for i in must_use_plants), 'must_use')

# 5. Plants that we cannot use
# x_{i} = 0    for all i in UI
mdl.addConstrs((x[i] == 0 for i in unavailable_plants), 'cant_use')

# Solve the model
mdl.optimize()
status = mdl.status
if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('The model is either infeasible or unbounded!')
    sys.exit(1)
elif status == GRB.OPTIMAL:
    print(f'The solution is optimal and the objective value is: {mdl.objVal:,.0f}')
else:
    print(f'Status is {status}. Investigate!')
    sys.exit(1)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 1602 rows, 1515 columns and 4516 nonzeros
Model fingerprint: 0xb4e4004c
Variable types: 0 continuous, 1515 integer (1515 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint cant_use[15] by 1.000000000

Presolve removed 101 rows and 101 columns
Presolve time: 0.02s
Presolved: 1501 rows, 1414 columns, 4214 nonzeros
Variable types: 0 continuous, 1414 integer (1414 binary)
Found heuristic solution: objective 1.149150e+08

Root relaxation: objective 6.390801e+07, 507 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  De

In [19]:
lanes = generate_outputs(plant_df, cust_df, dist, dmd, total_weighted_dist, x, y)

Selected plants are in: Baltimore, Phoenix, Memphis
Weighted Average Distance: 377.8
Total demand: 169,160
Percentages of demand within 400 miles of a plant: 64.08%
Percentages of demand within 800 miles of a plant: 94.46%
Percentages of demand within 1200 miles of a plant: 100.00%


In [20]:
plot_network(lanes, auto_open_map=False, title='Solution Map', open_map_in_cell=open_map_in_cell)

# Different Objective

- Because of the contract that the company has with the carriers, the transportation cost only depends on distance.
- Users are interested to see how considering transportation cost, changes the solution.
- For that, all we need to do is to change our objective function from weighted distance to cost.

Showing transportation cost between plant $i$ and customer $j$ with $c_{ij}$, the new objective will be:
$$ \sum_{ij} c_{ij}y_{ij}$$

In [ ]:
dist['Cost'] = dist['Distance'].apply(lambda x: max(cost_per_mile * x, min_cost))

# Objective: minimize total transportation cost
# sum_{ij} c_{ij}*y_{ij}
total_cost = gp.quicksum((dist.loc[i, j]['Cost'] * y[i, j]) for (i, j) in ij_set)
mdl.setObjective(total_cost, GRB.MINIMIZE)

# Solve the model
mdl.optimize()
status = mdl.status
if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('The model is either infeasible or unbounded!')
    sys.exit(1)
elif status == GRB.OPTIMAL:
    print(f'The solution is optimal and the objective value is: {mdl.objVal:,.0f}')
else:
    print(f'Status is {status}. Investigate!')
    sys.exit(1)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1602 rows, 1515 columns and 4516 nonzeros
Model fingerprint: 0xbebf2576
Variable types: 0 continuous, 1515 integer (1515 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

Loaded MIP start from previous solve with objective 79672.2

Presolve removed 101 rows and 101 columns
Presolve time: 0.01s
Presolved: 1501 rows, 1414 columns, 4214 nonzeros
Variable types: 0 continuous, 1414 integer (1414 binary)

Root relaxation: cutoff, 484 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      79672.1639 79672.1639  0.00%     -    0s

Explored 1 nodes (484 sim

In [ ]:
lanes = generate_outputs(plant_df, cust_df, dist, dmd, total_weighted_dist, x, y)

Selected plants are in: Baltimore, Phoenix, Memphis
Weighted Average Distance: 377.8
Total demand: 169,160
Percentages of demand within 400 miles of a plant: 64.08%
Percentages of demand within 800 miles of a plant: 94.46%
Percentages of demand within 1200 miles of a plant: 100.00%


In [ ]:
plot_network(lanes, auto_open_map=False, title='Solution Map', open_map_in_cell=open_map_in_cell)